# Load dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
EnergySet = pd.read_csv("data/energydata_complete.csv")
energydf = pd.DataFrame(data=EnergySet)

Quick look at what the dataset contains

In [ ]:
energydf.info()

# Split training and test data sets

In [3]:
from sklearn.model_selection import train_test_split

# Split into training and test data
energy_train, energy_test = train_test_split(energydf, test_size=0.2, random_state=12)

# Preprocessing
Cleaning the data and converting any data types

For showing off our preprocessing steps, a copy of the training data is used

In [20]:
preprocessing_set = energy_train.copy()

In [ ]:
# See if there are incorrect values from mix/max
# eg, negative humidity
pd.set_option("display.max_columns", 30)
preprocessing_set.describe()

## Clean data
Clean data for invalid values like negative humidity or unrealistic temperatures

In [4]:
def clean_data(min_value, max_value, columns, data):
    clean_data = data.copy()
    for col in columns:
        # Only keep rows where column date is within the min and max values
        clean_data = clean_data[ (clean_data[col] <= max_value)
                               & (clean_data[col] >= min_value)]
    return clean_data

Example of using the `clean_data` method, where we want the column `T1` to have values between 20 and 50

In [ ]:
clean_data(20,50,["T1"],preprocessing_set)

## Data Visualisation
Plot visualisations of data to see if we can gain any insight

### Histograms

In [ ]:
hists = preprocessing_set.hist(bins=100,figsize=(20,15))

Correlation between each temperature/humidity reading as they on the same sensor

In [ ]:
Ts = ["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9", "T_out"]
RHs = ["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9", "RH_out"]

In [ ]:
fig = plt.figure(figsize=(15,12))
fig.subplots_adjust(wspace=0.5)
for i in range(len(Ts)):
    plt.subplot(3,4,(i+1))
    plt.scatter(preprocessing_set[Ts[i]], preprocessing_set[RHs[i]], alpha=0.02)
    plt.xlabel(Ts[i])
    plt.ylabel(RHs[i])

## Convert date timestamp to numeric values

In [5]:
from datetime import datetime

def parse_timestamp(timestamp):
    return datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")

A testing dataframe is populated here to show examples of each conversion

In [6]:
time_test = preprocessing_set.copy()
time_test = time_test.loc[:, "date":"Appliances"]

### Numeric
Turn datetime into basic numeric forms, for example monday=0, tuesday=1...

In [7]:
def get_numeric_time(timestamp):
    dt = parse_timestamp(timestamp)
    hour = dt.hour
    day = dt.weekday()
    
    return hour,day

Example numeric time conversion

In [8]:
time_test[["hour_num", "day_num"]] = preprocessing_set["date"].map(get_numeric_time).apply(pd.Series)

In [9]:
time_test[["date","hour_num", "day_num"]].head(5)

,date,hour_num,day_num
11928,2016-04-03 13:00:00,13,6
11089,2016-03-28 17:10:00,17,0
8623,2016-03-11 14:10:00,14,4
15433,2016-04-27 21:10:00,21,2
11827,2016-04-02 20:10:00,20,5


### Cyclic
Due to the nature of time being cyclical, we want to keep this characteristic for our learning model. A way to do this is using sine and cosine transformations.
https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/

In [10]:
def get_cyclic_time(timestamp):
    dt = parse_timestamp(timestamp)
    
    hour_sin = np.sin(2*np.pi*dt.hour/24)
    hour_cos = np.cos(2*np.pi*dt.hour/24)
    day_sin = np.sin(2*np.pi*dt.weekday()/7)
    day_cos = np.cos(2*np.pi*dt.weekday()/7)
    
    return hour_sin, hour_cos, day_sin, day_cos

Example cyclic time conversion

In [11]:
time_test[["hour_sin", "hour_cos", "day_sin", "day_cos"]] = preprocessing_set["date"].map(get_cyclic_time).apply(pd.Series)

In [12]:
time_test[["date", "hour_sin", "hour_cos", "day_sin", "day_cos"]].head(5)

,date,hour_sin,hour_cos,day_sin,day_cos
11928,2016-04-03 13:00:00,-0.258819,-0.965926,-0.781831,0.623490
11089,2016-03-28 17:10:00,-0.965926,-0.258819,0.000000,1.000000
8623,2016-03-11 14:10:00,-0.500000,-0.866025,-0.433884,-0.900969
15433,2016-04-27 21:10:00,-0.707107,0.707107,0.974928,-0.222521
11827,2016-04-02 20:10:00,-0.866025,0.500000,-0.974928,-0.222521


### Seconds from midnight
A feature used by the paper is to extract the timestamp as the number of seconds from midnight (NSM) instead of using the actual hours of the day

In [13]:
def get_nsm(timestamp):
    dt = parse_timestamp(timestamp)
    midnight = dt.replace(hour=0, minute=0, second=0, microsecond=0)
    
    seconds = (dt - midnight).seconds
    
    return seconds
    
def get_cyclic_nsm(timestamp):
    seconds = get_nsm(timestamp)
    seconds_sin = np.sin(2*np.pi*seconds/(24*60*60))
    seconds_cos = np.cos(2*np.pi*seconds/(24*60*60))
    
    return seconds_sin, seconds_cos

Example NSM conversion

In [14]:
time_test[["NSM"]] = preprocessing_set["date"].map(get_nsm).apply(pd.Series)
time_test[["NSM_sin", "NSM_cos"]] = preprocessing_set["date"].map(get_cyclic_nsm).apply(pd.Series)

In [15]:
time_test[["NSM", "NSM_sin", "NSM_cos"]].head(5)

,NSM,NSM_sin,NSM_cos
11928,46800,-0.258819,-0.965926
11089,61800,-0.976296,-0.216440
8623,51000,-0.537300,-0.843391
15433,76200,-0.675590,0.737277
11827,72600,-0.843391,0.537300


### Categorical day of the week
As NSM does not convey any information about the day of the week, we can also convert the date into categorical values for each day of the week

In [16]:
def get_categorical_day(timestamp):
    dt = parse_timestamp(timestamp)
    day_num = dt.weekday()

    # Categories in order: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday
    categories = [0,0,0,0,0,0,0]
    categories[day_num] = 1
    
    return tuple(categories)

In [17]:
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
time_test[days_of_week] = preprocessing_set["date"].map(get_categorical_day).apply(pd.Series)

In [18]:
time_test[["date"] + days_of_week].head(5)

,date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
11928,2016-04-03 13:00:00,0,0,0,0,0,0,1
11089,2016-03-28 17:10:00,1,0,0,0,0,0,0
8623,2016-03-11 14:10:00,0,0,0,0,1,0,0
15433,2016-04-27 21:10:00,0,0,1,0,0,0,0
11827,2016-04-02 20:10:00,0,0,0,0,0,1,0


Comparison of different timestamp metrics.

In [19]:
time_correlation = time_test.corr()
time_correlation["Appliances"].map(abs).sort_values(ascending=False)

Appliances    1.000000
NSM_sin       0.259581
hour_sin      0.235093
hour_cos      0.233518
NSM           0.215358
hour_num      0.215016
NSM_cos       0.205434
day_sin       0.054228
Monday        0.052503
Tuesday       0.043997
Saturday      0.032062
Wednesday     0.030169
Friday        0.026120
Thursday      0.024768
Sunday        0.010392
day_cos       0.008363
day_num       0.003635
Name: Appliances, dtype: float64

### Plot histograms for all columns

In [ ]:
hists = preprocessing_set.hist(bins=100,figsize=(20,15))

## Feature selection

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#scaling = Pipeline([
#    ("scaler", StandardScaler())
#])



In [ ]:
preprocessing_set[["NSM_sin", "NSM_cos"]] = preprocessing_set["date"].map(get_cyclic_nsm).apply(pd.Series)
preprocessing_set[["hour_sin", "hour_cos", "day_sin", "day_cos"]] = preprocessing_set["date"].map(get_cyclic_time).apply(pd.Series)
preprocessing_set[["hour_num", "day_num"]] = preprocessing_set["date"].map(get_numeric_time).apply(pd.Series)
preprocessing_set[days_of_week] = preprocessing_set["date"].map(get_categorical_day).apply(pd.Series)

### Visualise feature correlation

In [ ]:
correlation = preprocessing_set.corr().abs()

In [ ]:
# Visualisation code adapted from https://seaborn.pydata.org/examples/many_pairwise_correlations.html
import seaborn as sns

# Mask upper half as it is just the mirror of the lower half
mask = np.zeros_like(correlation, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Colormap
cmap = sns.diverging_palette(220,10,as_cmap=True)

plt.figure(figsize=(12,12))
sns.heatmap(correlation, mask=mask, cmap=cmap, center=0, square=True, linewidths=0.5)

In [ ]:
correlation

In [ ]:
correlation["Appliances"].map(abs).sort_values(ascending=False)

### Data selection

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(.8* (1-.8)))
#sel.fit_transform(energy_train.drop("date", axis=1))

In [ ]:
t = preprocessing_set.drop("date", axis=1)
sel.fit_transform(t)
labels= [t.columns[x] for x in sel.get_support(indices=True) if x]
df = pd.DataFrame(sel.fit_transform(t), columns=labels)


In [ ]:
def VarianceThreshold_selector(data):

    #Select Model
    selector = VarianceThreshold(0.2) #Defaults to 0.0, e.g. only remove features with the same value in all samples

    #Fit the Model
    selector.fit(data)
    features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
    features = [column for column in data[features]] #Array of all nonremoved features' names

    #Format and Return
    selector = pd.DataFrame(selector.transform(data))
    selector.columns = features
    return selector

In [ ]:
VarianceThreshold_selector(t)

In [ ]:
def step_forward_selection(model):
    

### Check correlations

In [ ]:
cm = energy.corr()
cm["Appliances"].map(abs).sort_values(ascending=False)

## Model selection and training

### Pipelines
All the done above can be converted into pipelines. This allows us to modularly choose the exact stages to go through.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

#### Cleaning pipeline

In [ ]:
class DataCleaner(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self # No fitting for cleaning step
        
    def transform(self, X):
        # Temperature columns
        Tcols= ["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9", "T_out", "Tdewpoint"]
        
        # Humidity columns
        RHcols = ["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9", "RH_out"]
        
        # Non negative columns
        Ncols = ["lights", "Windspeed", "Visibility", "Press_mm_hg"]
        
        X = self.clean_data(-50,50,Tcols,X) # Clean temperatures
        X = self.clean_data(0,100,RHcols,X) # Clean humidity
        X = self.clean_data(0,1100,Ncols,X) # Clean non negative values, 1080 was the maximum value of any data cell from peeking at data with DataFrame.describe()
        
        return X
    

    def clean_data(self, min_value, max_value, columns, data):
        clean_data = data.copy()
        for col in columns:
            clean_data = clean_data[ (clean_data[col] <= max_value)
                                   & (clean_data[col] >= min_value)]
        return clean_data

#### Datetime transformation pipeline

In [ ]:
# This transformer takes the DataFrame and converts the date timestamp
# The date column is then dropped
class DatetimeTransformer(BaseEstimator, TransformerMixin):
        
    def fit(self, X, y=None):
        return self # No fitting in transformer
    
    def transform(self, X):
        cyclic_columns = ["hour_sin", "hour_cos", "day_sin", "day_cos"]    
        X[cyclic_columns] = self.mapTransformation(X, get_cyclic_time)
        X.drop(["day_sin", "day_cos"], axis=1)
        
        categorical_columns = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] 
        X[categorical_columns] = self.mapTransformation(X, get_categorical_day)
        
        return self.dropDate(X)
        
    def mapTransformation(self, X, mapper):
        return X["date"].map(mapper).apply(pd.Series)
    
    def dropDate(self, X):
        return X.drop("date", axis=1)

The `DFNPConverter` converts between a DataFrame and numpy array.

In [ ]:
class DFNPConverter(BaseEstimator, TransformerMixin):
    def __init__(self, toDF=True, DF_index=None, DF_columns=None):
        self.toDF = toDF
        
        # If converting back to DF, need index and columns of original DataFrame
        self.DF_index = DF_index
        self.DF_columns = DF_columns
        
    def fit(self, X, y=None):
        return self # No fitting for df/np conversion
    
    def transform(self, X):
        if self.toDF:
            return pd.DataFrame(X, index=self.DF_index, columns=self.DF_columns)
        
        else:
            return X.values

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

preprocessing_pipeline = Pipeline([
    ("cleaning", DataCleaner()),
    ("time_conversion", DatetimeTransformer()),
    ("feature_scaling", StandardScaler())
])

linreg_pipeline = Pipeline([
    ("model", LinearRegression())
])

polyreg_pipeline = Pipeline([
    ("poly", PolynomialFeatures(degree=2)),
    ("model", LinearRegression())
])

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

class Trainer():
    def __init__(self, preprocess, model, input, output, kfold=5):
        self.model = model
        self.preprocess = preprocess
        self.input = input
        self.output = output
        self.kfold = kfold
        
    
    def train(self, params=False):
        processed_input = self.preprocess.fit_transform(self.input)
        
        if not params:
            self.model.fit(processed_input, self.output)
            return self.cross_validate(processed_input, self.model)
        else:
            self.model.fit(processed_input, self.output)
            best_model = self.get_model().best_estimator_
            return self.cross_validate(processed_input, model=best_model)
    
    def get_model(self):
        return self.model.named_steps["model"]
        
    def cross_validate(self, processed_input, model):
        prediction = cross_val_predict(model, processed_input, self.output, cv=self.kfold)
        rmse = np.sqrt(mean_squared_error(self.output, prediction))
        r2 = r2_score(self.output, prediction)
        
        return rmse,r2
    

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

# Drop output column and random variables column
input = energy_train.drop(["Appliances", "rv1", "rv2"], axis=1)
output = energy_train["Appliances"]

In [ ]:
lin_trainer = Trainer(preprocessing_pipeline, linreg_pipeline, input, output, kfold=5)
lin_rmse, lin_r2 = lin_trainer.train()
lin_model = lin_trainer.get_model()

In [ ]:
lin_rmse

In [ ]:
lin_r2

### Ridge regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_pipeline = Pipeline([
    ("model", GridSearchCV(Ridge(), [{"alpha": [0.1,0.5,1,2,5,10,20]}]))
])

In [ ]:
ridge_trainer = Trainer(preprocessing_pipeline, ridge_pipeline, input, output, kfold=5)
ridge_rmse, ridge_r2 = ridge_trainer.train(params=True)
ridge_model = ridge_trainer.get_model().best_estimator_

In [ ]:
ridge_rmse

In [ ]:
ridge_r2

In [ ]:
ridge_trainer.get_model().best_params_

### Polynomial Regression

In [ ]:
poly_trainer = Trainer(preprocessing_pipeline, polyreg_pipeline, input, output, kfold=5)
poly_rmse, poly_r2 = poly_trainer.train()
poly_model = poly_trainer.get_model()

In [ ]:
poly_rmse

In [ ]:
poly_r2

### Polynomial ridge regression

In [ ]:
polyridge_pipeline = Pipeline([
    ("poly", PolynomialFeatures(degree=2)),
    ("model", Ridge(alpha=10))
])

In [ ]:
polyridge_trainer = Trainer(preprocessing_pipeline, polyridge_pipeline, input, output, kfold=5)
polyridge_rmse, polyridge_r2 = polyridge_trainer.train()
polyridge_model = ridge_trainer.get_model()

In [ ]:
polyridge_rmse

In [ ]:
polyridge_r2

In [ ]:
# use log
# https://datascience.stackexchange.com/questions/5000/proper-way-of-fighting-negative-outputs-of-a-regression-algorithms-where-output
# l = list(map(np.exp, ridge_res))
# output.map(np.log)

In [ ]:
plt.scatter(l, output, alpha=0.1)
plt.xlabel("Predicted value")
plt.ylabel("True value")

In [ ]:
ls = list(map(np.exp, lasso_res))

In [ ]:
plt.scatter(l, output, alpha=0.1)
plt.xlabel("Predicted value")
plt.ylabel("True value")